
### Folium ile Etkileşimli Görsel Analiz

#### **Amaçlar**

Bu laboratuvar aşağıdaki görevleri içermektedir:

* **GÖREV 1:** Tüm fırlatma noktalarını bir harita üzerinde işaretleyin
* **GÖREV 2:** Her fırlatma noktasına ait başarılı/başarısız fırlatmaları harita üzerinde gösterin
* **GÖREV 3:** Bir fırlatma noktasının yakın çevresindeki konumlara olan mesafesini hesaplayın

Yukarıdaki görevleri tamamladıktan sonra, fırlatma noktalarıyla ilgili bazı coğrafi desenleri gözlemleyebilmelisiniz.

---

Öncelikle bu laboratuvar için gerekli Python kütüphanelerini içe aktaralım:


In [1]:
import folium
import pandas as pd

In [2]:
# folium MarkerCluster eklentisini içe aktar (birden fazla işaretçiyi gruplayarak gösterir)
from folium.plugins import MarkerCluster

# folium MousePosition eklentisini içe aktar (fare imlecinin harita üzerindeki koordinatlarını gösterir)
from folium.plugins import MousePosition

# folium DivIcon bileşenini içe aktar (harita üzerine yazı ya da özel ikon yerleştirmek için kullanılır)
from folium.features import DivIcon


## Görev 1: Tüm fırlatma noktalarını bir harita üzerinde işaretleyin


İlk olarak, her fırlatma noktasının konumunu, o noktanın enlem (latitude) ve boylam (longitude) koordinatlarını kullanarak haritaya eklemeyi deneyelim.

**spacex\_launch\_geo.csv** adlı aşağıdaki veri kümesi, her fırlatma noktası için eklenmiş enlem ve boylam bilgilerini içeren genişletilmiş bir veri kümesidir.


In [3]:
url= 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(url)

Şimdi, her bir fırlatma noktasının koordinatlarına göz atabilirsin.


In [4]:
# Gerekli alt sütunları seç: 'Launch Site', 'Lat' (enlem), 'Long' (boylam), 'class' (başarı/basarıszlık)
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Her bir fırlatma noktasının ilk kaydını al (bir kere temsil etmesi için)
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

# Yalnızca fırlatma noktası adı, enlem ve boylam sütunlarını bırak
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Sonuçları görüntüle
launch_sites_df


,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745



Yukarıdaki koordinatlar yalnızca düz sayılardır ve bu sayılar, fırlatma noktalarının nerede olduğu hakkında sezgisel bir fikir vermez.
Eğer coğrafya bilginiz çok iyiyse, bu sayıları doğrudan zihninizde yorumlayabilirsiniz. Değilse sorun değil, bu konumları harita üzerinde işaretleyerek görselleştirelim.

İlk olarak, bir **Folium Harita (Map)** nesnesi oluşturmamız gerekiyor. Bu haritanın başlangıç merkezi olarak **Houston, Texas’taki NASA Johnson Uzay Merkezi**'ni belirleyeceğiz.


In [5]:
# Başlangıç konumu olarak NASA Johnson Space Center'ın koordinatlarını tanımla
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Bu koordinatları kullanarak Folium harita nesnesi oluştur (zoom seviyesini 10 olarak ayarla)
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Belirli bir koordinata metin etiketi ile birlikte vurgulanmış bir daire alanı eklemek için folium.Circle kullanabiliriz. Örneğin,

In [6]:
# NASA Johnson Space Center koordinatlarında, üzerinde adı gösterilen turuncu bir daire oluştur
circle = folium.Circle(
    nasa_coordinate,        # Dairenin merkezi koordinatları
    radius=1000,            # Dairenin yarıçapı (metre cinsinden)
    color='#d35400',        # Dairenin sınır rengi (turuncu)
    fill=True              # Dairenin dolu olması
).add_child(folium.Popup('NASA Johnson Space Center'))  # Üzerine tıklandığında isim gösterecek popup

# Aynı koordinatta, üzerine 'NASA JSC' yazan bir metin etiketi ekle
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',  # Yazı stili ve metin
    )
)

# Haritaya daireyi ekle
site_map.add_child(circle)
# Haritaya yazı etiketini ekle
site_map.add_child(marker)




Ve Houston şehrinin yakınlarında küçük sarı bir daire bulmalısınız, ayrıca haritayı yakınlaştırdığınızda daha büyük bir daire görebilirsiniz.

Şimdi, `launch_sites` veri çerçevesindeki her fırlatma sitesi için bir daire ekleyelim.

YAPILACAKLAR:
Her fırlatma sitesi için `folium.Circle` ve `folium.Marker` nesneleri oluşturup, bu nesneleri site haritasına ekleyin.

**folium.Circle örneği:**

```python
folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))
```

**folium.Marker örneği:**

```python
folium.map.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'etiket',
    )
)
```



In [7]:
# Haritayı başlat

site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Her fırlatma sitesi için koordinatlara (Enlem, Boylam) dayalı bir daire (Circle) nesnesi ekle. Ayrıca, fırlatma sitesi adını bir açılır pencere (popup) etiketi olarak göster.

for index, row in launch_sites_df.iterrows():
    # Daireyi oluştur
    circle = folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#d35400',
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))

    # Metin etiketi oluştur
    marker = folium.map.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site']
        )
    )

    # Haritaya daireyi ve etiketi ekle
    site_map.add_child(circle)
    site_map.add_child(marker)

# Haritayı göster
site_map

Şimdi, işaretlenmiş alanları yakınlaştırıp uzaklaştırarak haritayı keşfedebilirsiniz ve aşağıdaki soruları yanıtlamaya çalışabilirsiniz:

* Tüm fırlatma alanları Ekvator çizgisine yakın mı?
* Tüm fırlatma alanları kıyıya çok yakın mı?
* Ayrıca, bulgularınızı açıklamaya çalışınız.


# Görev 2: Her bir fırlatma sitesindeki başarılı/başarısız fırlatmaları harita üzerinde işaretleyin


Şimdi, her bir site için fırlatma sonuçlarını haritaya ekleyerek haritayı geliştirmeye çalışalım ve hangi sitelerin yüksek başarı oranına sahip olduğunu görelim. Hatırlarsanız, spacex\_df veri çerçevesi detaylı fırlatma kayıtlarını içeriyor ve class sütunu bu fırlatmanın başarılı olup olmadığını gösteriyor.


In [8]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Şimdi, tüm fırlatma kayıtları için işaretçiler oluşturalım. Eğer bir fırlatma başarılı ise (class=1), yeşil bir işaretçi kullanacağız; başarısız ise (class=0) kırmızı işaretçi kullanacağız.

Unutmayın, bir fırlatma sadece dört fırlatma sitesinden birinde gerçekleşir, yani birçok fırlatma kaydının aynı koordinatlara sahip olması muhtemeldir. Aynı koordinata sahip birçok işaretçinin bulunduğu bir haritayı sadeleştirmek için MarkerCluster iyi bir yöntem olabilir.

Öncelikle bir MarkerCluster nesnesi oluşturalım.


In [9]:
marker_cluster = MarkerCluster()


**Yapılacaklar:** spacex\_df veri çerçevesinde, class değerine göre işaretçi renklerini saklamak için marker\_color adında yeni bir sütun oluşturun.


In [10]:
# `class` sütununun değerini kontrol eden bir fonksiyon uygulayın
# Eğer class=1 ise, marker_color değeri yeşil olacak
# Eğer class=0 ise, marker_color değeri kırmızı olacak

def assign_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)


**Yapılacaklar:** spacex\_df veri çerçevesindeki her bir fırlatma sonucuna karşılık, marker\_cluster'a bir folium.Marker ekleyin.


In [12]:
# MarkerCluster nesnesini mevcut site_map haritasına ekle
site_map.add_child(marker_cluster)

# spacex_df veri çerçevesindeki her satır için döngü
for index, record in spacex_df.iterrows():
    # Her fırlatmanın koordinatlarını kullanarak bir Marker nesnesi oluştur
    # Marker'ın icon özelliği, fırlatma sonucuna göre marker_color sütunundan belirleniyor
    # color parametresi marker arka plan rengini,
    # icon_color parametresi ise marker içindeki ikon rengini ayarlıyor
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"Launch Result Class: {record['class']}",
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    # Oluşturulan marker'ı marker_cluster'a ekle
    marker_cluster.add_child(marker)

# Haritayı çıktı olarak göster
site_map


Marker kümelerindeki renk etiketli işaretçilerden, hangi fırlatma alanlarının nispeten yüksek başarı oranına sahip olduğunu kolayca anlayabilmelisiniz.


# GÖREV 3: Bir fırlatma alanı ile çevresindeki noktalar arasındaki mesafeleri hesaplayın


Sonraki adımda, fırlatma alanlarının çevresini keşfetmemiz ve analiz etmemiz gerekiyor.

Öncelikle haritaya bir MousePosition (fare konumu) aracı ekleyelim. Böylece harita üzerinde herhangi bir noktanın üzerine fare ile geldiğinizde koordinatlarını görebilirsiniz. Bu sayede haritayı incelerken, ilgi alanı olan herhangi bir noktanın (örneğin, demiryolu) koordinatlarını kolayca bulabilirsiniz.


In [14]:
# Koordinatların ondalık basamak sayısını ayarlamak için formatlama fonksiyonu tanımlanıyor.
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# MousePosition nesnesi oluşturuluyor.
# Bu nesne fare ile harita üzerinde gezinirken enlem ve boylam bilgisini gösterir.
mouse_position = MousePosition(
    position='topright',      # Harita üzerinde fare koordinatlarının gösterileceği yer (sağ üst köşe).
    separator=' Long: ',      # Enlem ve boylam arasındaki ayracı belirtir.
    empty_string='NaN',       # Koordinat bilgisi yoksa gösterilecek metin.
    lng_first=False,          # Enlem (Lat) bilgisinin önce gösterilmesi, sonra boylam (Lng) gösterilmesi için False.
    num_digits=20,            # Koordinatların gösterileceği ondalık basamak sayısı.
    prefix='Lat:',            # Enlem bilgisinin önüne eklenecek metin.
    lat_formatter=formatter,  # Enlem bilgisinin nasıl formatlanacağı (ondalık basamak sayısı).
    lng_formatter=formatter,  # Boylam bilgisinin nasıl formatlanacağı.
)

# Oluşturulan MousePosition bileşeni haritaya ekleniyor.
site_map.add_child(mouse_position)

# Harita nesnesi gösteriliyor.
site_map



**"Şimdi bir fırlatma sitesine yakınlaştırın ve çevresini keşfedin; yakınlarda kolayca herhangi bir demiryolu, karayolu, sahil şeridi vb. bulup bulamayacağınıza bakın. Fareyi bu noktalara getirerek (sol üstte gösterilen) koordinatlarını not edin ve bunları fırlatma sitesine olan mesafeyi hesaplamak için kullanın."**



In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # Dünyanın yaklaşık yarıçapı (km cinsinden)
    R = 6373.0

    # Derece cinsinden verilen koordinatları radyana çeviriyoruz
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Enlem ve boylam farkları
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formülünün 'a' kısmı
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

    # Haversine formülünün 'c' kısmı
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # İki nokta arasındaki mesafe
    distance = R * c

    return distance


**"Yapılacaklar:** MousePosition kullanarak en yakın sahil noktasını işaretleyin ve sahil noktası ile fırlatma sitesi arasındaki mesafeyi hesaplayın."



In [41]:
launch_site_lat = 28.5632
launch_site_lon = -80.57685

# Sahil noktası koordinatları
coastline_lat = 28.56323
coastline_lon = -80.56704

# Mesafeyi hesapla
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [42]:
# Harita oluştur
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# En yakın sahil noktasına marker ekle ve mesafeyi göster
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)

site_map.add_child(distance_marker)

**Yapılacak:** Fırlatma sitesi ile seçilen sahil noktası arasında bir PolyLine (çoklu çizgi) çiz.


In [43]:
# PolyLine çizmek için koordinat listesi
line_coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

# PolyLine objesi oluştur
lines = folium.PolyLine(locations=line_coordinates, weight=2, color='blue')

# Haritaya ekle
site_map.add_child(lines)

* En yakın şehir, demiryolu, otoyol vb. için mesafeyi gösteren bir işaretçi (marker) oluştur.
* Bu işaretçi ile fırlatma sitesi arasında bir çizgi çiz.

In [44]:
# En yakın şehir koordinatları
city_lat = 29.7604
city_lon = -95.3698

# Mesafe hesapla
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Şehir marker'ı ve mesafe etiketi
city_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#1f77b4;"><b>{:10.2f} KM</b></div>'.format(distance_city),
    )
)
site_map.add_child(city_marker)

# Fırlatma sitesi ile şehir arasında PolyLine çiz
city_line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]], weight=2, color='green')
site_map.add_child(city_line)




**Proksimite (yakın çevre) mesafelerine çizgi çizdikten sonra, aşağıdaki soruları kolayca cevaplayabilirsiniz:**

* Fırlatma alanları demiryollarına yakın mı?
* Fırlatma alanları otoyollara yakın mı?
* Fırlatma alanları sahil şeridine yakın mı?
* Fırlatma alanları şehirlerden belirli bir mesafede mi bulunuyor?

**Ayrıca, bulgularınızı açıklamaya çalışın.**

